In [5]:
from pylablib.devices import Thorlabs
import time

# List all Kinesis devices (serial number, model)
devices = Thorlabs.list_kinesis_devices()
devices

[('37004276', 'APT Filter Flipper'),
 ('37004214', 'APT Filter Flipper'),
 ('68000239', 'Solenoid Controller'),
 ('37004212', 'APT Filter Flipper'),
 ('37004216', 'APT Filter Flipper')]

In [9]:
import clr
import time
from System import UInt32  # not strictly needed for read, but harmless

# --- Load the exact DLLs like in your sample ---
clr.AddReference(r"C:\Program Files\Thorlabs\Kinesis\Thorlabs.MotionControl.DeviceManagerCLI.dll")
clr.AddReference(r"C:\Program Files\Thorlabs\Kinesis\Thorlabs.MotionControl.GenericMotorCLI.dll")
clr.AddReference(r"C:\Program Files\Thorlabs\Kinesis\Thorlabs.MotionControl.FilterFlipperCLI.dll")

from Thorlabs.MotionControl.DeviceManagerCLI import DeviceManagerCLI
from Thorlabs.MotionControl.FilterFlipperCLI import FilterFlipper

# Your labels -> serials
LABELS = {
    "Probe": "37004276",
    "PumpA": "37004216",
    "PumpB": "37004214",
    "PumpC": "37004212",
}

def _read_position(device):
    """Return 1 or 2 (0 may appear transient while moving). Try common getters."""
    # Preferred on many builds:
    try:
        return int(device.GetPosition())
    except Exception:
        pass
    # Property on some builds:
    try:
        return int(device.Position)
    except Exception:
        pass
    # Status API on others:
    try:
        return int(device.Status.Position)
    except Exception:
        pass
    # Fallback via GetStatus():
    try:
        return int(device.GetStatus().Position)
    except Exception:
        pass
    raise RuntimeError("No usable position getter on this DLL build")

def read_one(name, serial_no):
    # Build list once outside the loop is fine too; doing here keeps function standalone
    DeviceManagerCLI.BuildDeviceList()

    device = FilterFlipper.CreateFilterFlipper(serial_no)
    try:
        device.Connect(serial_no)

        # Ensure settings are initialized
        if not device.IsSettingsInitialized():
            device.WaitForSettingsInitialized(10000)

        # Start polling & enable (as in your sample)
        try:
            device.StartPolling(250)
            time.sleep(0.25)
        except Exception:
            pass  # some builds may not require/offer polling
        try:
            device.EnableDevice()
            time.sleep(0.25)
        except Exception:
            pass

        pos = _read_position(device)   # 1 or 2 when settled
        state = "ON" if pos == 1 else "OFF"   # swap if your optics define 2=ON
        print(f"{name} ({serial_no}): {state} [pos={pos}]")

    finally:
        try:
            device.StopPolling()
        except Exception:
            pass
        try:
            device.Disconnect()
        except Exception:
            pass

def main():
    for name, sn in LABELS.items():
        read_one(name, sn)

if __name__ == "__main__":
    main()


Probe (37004276): ON [pos=1]
PumpA (37004216): ON [pos=1]
PumpB (37004214): ON [pos=1]
PumpC (37004212): OFF [pos=2]


In [11]:
import clr, time
from System import UInt32

# --- Kinesis DLLs (same as your working reader) ---
clr.AddReference(r"C:\Program Files\Thorlabs\Kinesis\Thorlabs.MotionControl.DeviceManagerCLI.dll")
clr.AddReference(r"C:\Program Files\Thorlabs\Kinesis\Thorlabs.MotionControl.GenericMotorCLI.dll")
clr.AddReference(r"C:\Program Files\Thorlabs\Kinesis\Thorlabs.MotionControl.FilterFlipperCLI.dll")

from Thorlabs.MotionControl.DeviceManagerCLI import DeviceManagerCLI
from Thorlabs.MotionControl.FilterFlipperCLI import FilterFlipper

LABELS = {
    "Probe": "37004276",
    "PumpA": "37004216",
    "PumpB": "37004214",
    "PumpC": "37004212",
}

def _read_pos(dev):
    # Try the getters your environment supports
    for getter in (
        lambda d: int(d.Position),
        lambda d: int(d.Status.Position),
        lambda d: int(d.GetStatus().Position),
    ):
        try:
            return getter(dev)
        except Exception:
            pass
    raise RuntimeError("No usable position getter on this DLL build")

def _set_pos(dev, pos):
    # Prefer SetPosition(UInt32, timeout); fallback to other signatures
    for mover in (
        lambda d, p: d.SetPosition(UInt32(p), 60000),
        lambda d, p: d.SetPosition(UInt32(p)),
        lambda d, p: d.MoveTo(UInt32(p)),
    ):
        try:
            mover(dev, pos); return
        except Exception:
            continue
    raise RuntimeError("No compatible move method found for this DLL build")

def toggle_flip(name):
    sn = LABELS.get(name)
    if not sn:
        print(f"Unknown flip: {name}"); return

    DeviceManagerCLI.BuildDeviceList()
    dev = FilterFlipper.CreateFilterFlipper(sn)
    try:
        dev.Connect(sn)
        if not dev.IsSettingsInitialized():
            dev.WaitForSettingsInitialized(10000)

        # Poll/enable if available (some builds require these)
        try: dev.StartPolling(250); time.sleep(0.25)
        except Exception: pass
        try: dev.EnableDevice();   time.sleep(0.25)
        except Exception: pass

        cur = _read_pos(dev)          # 1 or 2 (0 only while moving)
        new_pos = 2 if cur == 1 else 1
        _set_pos(dev, new_pos)

        # Read back until it latches to 1 or 2
        for _ in range(80):           # ~4s max
            pos = _read_pos(dev)
            if pos in (1, 2):
                break
            time.sleep(0.05)

        state = "ON" if pos == 1 else "OFF"   # swap if your optics use 2=ON
        print(f"{name} ({sn}) toggled to: {state} [pos={pos}]")

    finally:
        try: dev.StopPolling()
        except Exception: pass
        try: dev.Disconnect()
        except Exception: pass

# Example:
toggle_flip("Probe")

Probe (37004276) toggled to: ON [pos=1]
